In [230]:
class SodukoData:
    def __init__(self):
        self.__data = [[None for i in range(9)] for i in range(9)]
        self.__data_transposed = [[None for i in range(9)] for i in range(9)]

    def get_row(self, row):
        return self.__data[row]

    def get_column(self, col):
        return self.__data_transposed[col]

    # Extract the full subset a certain cell is in
    def get_subset_of_cell(self, row, col):
        subset_y = (row // 3) * 3
        subset_x = (col // 3) * 3
        subset_x_end = subset_x + 3
        for i in range(3):
            yield self.__data[subset_y + i][subset_x:subset_x_end]

    def set_cell(self, row, col, val):
        self.__data[row][col] = val
        self.__data_transposed[col][row] = val

    def get_cell(self, row, col):
        return self.__data[row][col]

    # Check if an specific cell is valid for a certain value.
    # This means, the value cannot appear in the same row, column and subset.
    def valid_location(self, row, col, val):
        # if value is in row
        if val in self.get_row(row):
            return False
        # if value is in column
        if val in self.get_column(col):
            return False
        # if value is in subset
        for subset_row in self.get_subset_of_cell(row, col):
            if val in subset_row:
                return False
        return True

    # Export the soduko into a well known format
    def export(self):
        return "".join('.' if e is None else str(e) for row in self.__data for e in row)

    # Import an soduko from a well known format
    def load(self, data):
        for i,c in enumerate(data):
            self.set_cell(i // 9, i % 9, int(c) if c != '.' else None)

In [205]:
# Create a new empty soduko
s1 = SodukoData()
# Export should say it is empty
assert(s1.export() == '.' * 81)
# Fill two example cells
s1.set_cell(0,0,1)
s1.set_cell(8,8,1)
# Export should no include the changed cells
assert(s1.export() == f"1{'.' * 79}1")

In [206]:
# Generate example soduko
s1 = SodukoData()
s1.set_cell(0,0,1)
s1.set_cell(8,8,1)
# Create new soduko for loading the first one
s2 = SodukoData()
# Load the first soduko into the second one
s2.load(s1.export())
# Both sodukos should be the same
assert(s1.export() == s2.export())

In [278]:
# Benchmark performance of functions

s1 = SodukoData()

# First column, first subset, top row
s1.set_cell(0,0,1)
# First column, second subset, top row
s1.set_cell(3,0,2)

print(s1.export())

import timeit

def bench_func(func):
    r = timeit.Timer(func).timeit(number=1000) * 1000
    print("%.2fus %s" % (r,func.__name__))

@bench_func
def get_column():
    assert(list(s1.get_column(0)) == [1, None, None, 2, None, None, None, None, None])

@bench_func
def get_row():
    assert(list(s1.get_row(0)) == [1, None, None, None, None, None, None, None, None])

@bench_func
def get_subset_of_cell():
    assert(list(s1.get_subset_of_cell(0, 0)) == [[1, None, None], [None, None, None], [None, None, None]])

@bench_func
def valid_location_invalid_row():
    assert(s1.valid_location(0,1,1) == False)

@bench_func
def valid_location_invalid_column():
    assert(s1.valid_location(1,0,1) == False)

@bench_func
def valid_location_invalid_subset():
    assert(s1.valid_location(1,1,1) == False)

@bench_func
def valid_location_valid():
    assert(s1.valid_location(0,1,2) == True)

1..........................2.....................................................
0.17us get_column
0.17us get_row
0.67us get_subset_of_cell
0.12us valid_location_invalid_row
0.21us valid_location_invalid_column
0.77us valid_location_invalid_subset
0.83us valid_location_valid


In [133]:
""" Recurisve Backtrack Algorithm
=================================

This is probably the most simple but also most inefficient algorithm
to solve an soduko, as it is based in a brute-force like recursive
search.

The Solution for a random puzzle can be found within milliseconds or
multiple seconds.
"""
def recursive_backtrack_solve(s):
    # shortcut for finding an empty cell
    def find_empty_cell():
        # for every cell position
        for row in range(9):
            for col in range(9):
                # if cell empty, return position
                if s.get_cell(row, col) is None:
                    return (row, col)
        # return none if no empty cell found
        return None

    # find next empty cell
    cell = find_empty_cell()
    # if there is no empty cell, we are done
    if cell is None:
        return True
    # else unwrap position
    row, col = cell
    # try every possible candidate
    for candidate in range(1, 10):
        # check if it is a valid position
        if s.valid_location(row, col, candidate):
            # if so, fill cell
            s.set_cell(row, col, candidate)
            # try to solve the reaming empty cells
            if recursive_backtrack_solve(s):
                # if success with all cells, success here
                return True
            else:
                # else clear cell and retry next candidate
                s.set_cell(row, col, None)
    # no solution has been found
    return False

In [268]:
"""Optimized Backtrack Solve
============================

This algorithm creates an optimized dictonary of candidates for
each cell. This reduces the number of candidates that need to
be checked for every empty cell.

Additionally this allows the algorithm to fill trivial cells
with the only possible candidate if only one is left.

After this optimization, the normal backtrack solve is executed
with an optmized set of candidates.
"""
def optimized_recursive_backtrack_solve(s):
    # candidate table for each empty cell, index is f"{row}x{col}"
    # value an array of possible candidates
    def build_candidate_table():
        candidate_table = {}
        # for every cell
        for row in range(9):
            for col in  range(9):
                # if that cell is empty
                if s.get_cell(row, col) is None:
                    # store all valid candidates for that cell
                    candidate_table[(row, col)] = [c for c in range(1, 10) if s.valid_location(row, col, c)]
                    
        # optimize candidate table
        # missing steps:
        # - remove candidate from same subsection if there is only one row/column possible

        # check row for candidate that is only possible in that specific cell
        for row in range(9):
            # check if there is an candidate which only has one position
            for candidate in range(1, 10):
                possible_cells = []
                for col in range(9):
                    pos = (row, col)
                    if pos in candidate_table and candidate in candidate_table[pos]:
                        possible_cells.append(pos)
                if len(possible_cells) == 1:
                    candidate_table[possible_cells[0]] = [candidate]
        # check column for candidate that is only possible in that specific cell
        for col in range(9):
            for candidate in range(1, 10):
                possible_cells = []
                for row in range(9):
                    pos = (row, col)
                    if pos in candidate_table and candidate in candidate_table[pos]:
                        possible_cells.append(pos)
                if len(possible_cells) == 1:
                    candidate_table[possible_cells[0]] = [candidate]
        # # check for candidate in subset that is only possible in that specific cell
        subset_cells = {(row // 3, col // 3): [] for row in range(9) for col in range(9)}
        for subset_pos in subset_cells:
            subset_y, subset_x = [p * 3 for p in subset_pos]
            subset_cells[subset_pos] = [(row, col) for row in range(subset_y, subset_y + 3) for col in range(subset_x, subset_x + 3)]

        for subset_pos in subset_cells:
            for candidate in range(1,10):
                possible_subset_cells = []
                for subset_cell in subset_cells[subset_pos]:
                    if subset_cell in candidate_table and candidate in candidate_table[subset_cell]:
                        possible_subset_cells.append(subset_cell)
                if len(possible_subset_cells) == 1:
                    candidate_table[possible_subset_cells[0]] = [candidate]

        # return final candidate table
        return candidate_table

    # fill all trivial cells found in the candidate table
    # and return most latest candidate table
    def fill_trivial_cells():
        # build candidate table
        candidate_table = build_candidate_table()
        # repeat filling trivial cases until no trivial changes were made
        made_changes = True
        while made_changes:
            made_changes = False
            # for each empty cell listed in the candidate table
            for empty_cell in candidate_table:
                # load candidates
                candidates = candidate_table[empty_cell]
                # if there is only a single trivial candidate
                if len(candidates) == 1:
                    # load cell fill details
                    candidate = candidates.pop()
                    row, col = empty_cell
                    # fill the cell (validation was in the building process of the table)
                    s.set_cell(row, col, candidate)
                    # note that changes have been made
                    made_changes = True
            # rebuild candidate table if changes were made
            candidate_table = build_candidate_table()
            
        return candidate_table

    # recursive backtrack but with candidate table
    def recursive_backtrack_solve_with_candidate_table(s, ct):
        # shortcut for finding an empty cell
        def find_empty_cell():
            # for every cell position
            for row in range(9):
                for col in range(9):
                    # if cell empty, return position
                    if s.get_cell(row, col) is None:
                        return (row, col)
            # return none if no empty cell found
            return None

        # find next empty cell
        cell = find_empty_cell()
        # if there is no empty cell, we are done
        if cell is None:
            return True
        # else unwrap position
        row, col = cell
        # get candidates from lookup table
        candidates = ct[cell]
        # try every possible candidate
        for candidate in candidates:
            # check if it is a valid position
            if s.valid_location(row, col, candidate):
                # if so, fill cell
                s.set_cell(row, col, candidate)
                # try to solve the reaming empty cells
                if recursive_backtrack_solve_with_candidate_table(s, ct):
                    # if success with all cells, success here
                    return True
                # else clear cell and retry next candidate
                s.set_cell(row, col, None)
        # no solution has been found
        return False

    candidate_table = fill_trivial_cells()
    return recursive_backtrack_solve_with_candidate_table(s, candidate_table)

In [281]:
def benchmark(data, func, conv = None):
    import time

    result = {}
    # for every data entry to test
    for entry in data:
        # convert entry if needed
        runData = entry if conv is None else conv(entry)
        # execute function for entry and assert it is true
        # and measure time elapsed
        start = time.time()
        assert(func(runData))
        end = time.time()
        # store elapsed time
        result[entry] = end - start
    return dict(sorted(result.items(), key=lambda item: item[1]))

def conv_data_to_soduko(data):
    s = SodukoData()
    s.load(data)
    return s

In [283]:
# Benchmark easy puzzles
# Source: http://www.norvig.com/easy50.txt (converted by me to this format)
DATA = """..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..
2...8.3...6..7..84.3.5..2.9...1.54.8.........4.27.6...3.1..7.4.72..4..6...4.1...3
......9.7...42.18....7.5.261..9.4....5.....4....5.7..992.1.8....34.59...5.7......
.3..5..4...8.1.5..46.....12.7.5.2.8....6.3....4.1.9.3.25.....98..1.2.6...8..6..2.
.2.81.74.7....31...9...28.5..9.4..874..2.8..316..3.2..3.27...6...56....8.76.51.9.
1..92....524.1...........7..5...81.2.........4.27...9..6...........3.945....71..6
.43.8.25.6.............1.949....4.7....6.8....1.2....382.5.............5.34.9.71.
48...69.2..2..8..19..37..6.84..1.2....37.41....1.6..49.2..85..77..9..6..6.92...18
...9....2.5.1234...3....16.9.8.......7.....9.......2.5.91....5...7439.2.4....7...
..19....39..7..16..3...5..7.5......9..43.26..2......7.6..1...3..42..7..65....68..
...1254....84.....42.8......3.....95.6.9.2.1.51.....6......3.49.....72....1298...
.6234.75.1....56..57.....4.....948..4.......6..583.....3.....91..64....7.59.8326.
3..........5..9...2..5.4....2....7..16.....587.431.6.....89.1......67.8......5437
63..........5....8..5674.......2......34.1.2.......345.....7..4.8.3..9.29471...8.
....2..4...8.35.......7.6.2.31.4697.2...........5.12.3.49...73........1.8....4...
361.259...8.96..1.4......57..8...471...6.3...259...8..74......5.2..18.6...547.329
.5.8.7.2.6...1..9.7.254...6.7..2.3.15.4...9.81.3.8..7.9...762.5.6..9...3.8.1.3.4.
.8...5........3457....7.8.9.6.4..9.3..7.1.5..4.8..7.2.9.1.2....8423........1...8.
..35.29......4....1.6...3.59..251..8.7.4.8.3.8..763..13.8...1.4....2......51.48..
...........98.51...519.742.29.4.1.65.........14.5.8.93.267.958...51.36...........
.2..3..9....9.7...9..2.8..5..48.65..6.7...2.8..31.29..8..6.5..7...3.9....3..2..5.
..5.....6.7...9.2....5..1.78.415.......8.3.......928.59.7..6....3.4...1.2.....6..
.4.....5...19436....9...3..6...5...21.3...5.68...2...7..5...2....24367...3.....4.
..4..........3...239.7...8.4....9..12.98.13.76..2....8.1...8.539...4..........8..
36..2..89...361............8.3...6.24..6.3..76.7...1.8............418...97..3..14
5..4...6...9...8..64..2.........1..82.8...5.17..5.........9..84..3...6...6...3..2
..72564..4.......5.1..3..6....5.8.....8.6.2.....1.7....3..7..9.2.......4..63127..
..........79.5.18.8.......7..73.68..45.7.8.96..35.27..7.......5.16.3.42..........
.3.....8...9...5....75.92..7..1.5..8.2..9..3.9..4.2..1..42.71....2...8...7.....9.
2..17.6.3.5....1.......6.79....4.7.....8.1.....9.5....31.4.......5....6.9.6.37..2
.......8.8..7.1.4..4..2..3.374...9......3......5...321.1..6..5..5.8.2..6.8.......
.......85...21...996..8.1..5..8...16.........89...6..7..9.7..523...54...48.......
6.8.7.5.2.5.6.8.7...2...3..5...9...6.4.3.2.5.8...5...3..5...2...1.7.4.9.4.9.6.7.1
.5..1..4.1.7...6.2...9.5...2.8.3.5.1.4..7..2.9.1.8.4.6...4.1...3.4...7.9.2..6..1.
.53...79...97534..1.......2.9..8..1....9.7....8..3..7.5.......3..76412...61...94.
..6.8.3...49.7.25....4.5...6..317..4..7...8..1..826..9...7.2....75.4.19...3.9.6..
..5.8.7..7..2.4..532.....84.6.1.5.4...8...5...7.8.3.1.45.....916..5.8..7..3.1.6..
...9..8..128..64...7.8...6.8..43...75.......96...79..8.9...4.1...36..284..1..7...
....8....27.....54.95...81...98.64...2.4.3.6...69.51...17...62.46.....38....9....
...6.2...4...5...1.85.1.62..382.671...........194.735..26.4.53.9...2...7...8.9...
...9....2.5.1234...3....16.9.8.......7.....9.......2.5.91....5...7439.2.4....7...
38..........4..785..9.2.3...6..9....8..3.2..9....4..7...1.7.5..495..6..........92
...158.....2.6.8...3.....4..27.3.51...........46.8.79..5.....8...4.7.1.....325...
.1.5..2..9....1.....2..8.3.5...3...7..8...5..6...8...4.4.1..7.....7....6..3..4.5.
.8.....4....469...4.......7..59.46...7.6.8.3...85.21..9.......5...781....6.....1.
9.42....7.1..........7.65.....8...9..2.9.4.6..4...2.....16.7..........3.3....57.2
...7..8....6....31.4...2....24.7.....1..3..8.....6.29....8...7.86....5....2..6...
..1..7.9.59..8...1.3.....8......58...5..6..2...41......8.....3.1...2..79.2.7..4..
.....3.17.15..9..8.6.......1....7.....9...2.....5....4.......2.5..6..34.34.2.....
3..2........1.7...7.6.3.5...7...9.8.9...2...4.1.8...5...9.4.3.1...7.2........8..6"""

#print(benchmark(DATA.split("\n"), recursive_backtrack_solve, conv_data_to_soduko)) # ~ 31s

print(benchmark(DATA.split("\n"), optimized_recursive_backtrack_solve, conv_data_to_soduko)) # ~ 0.2s

{'361.259...8.96..1.4......57..8...471...6.3...259...8..74......5.2..18.6...547.329': 0.0, '.2.81.74.7....31...9...28.5..9.4..874..2.8..316..3.2..3.27...6...56....8.76.51.9.': 0.0009975433349609375, '....8....27.....54.95...81...98.64...2.4.3.6...69.51...17...62.46.....38....9....': 0.0009992122650146484, '48...69.2..2..8..19..37..6.84..1.2....37.41....1.6..49.2..85..77..9..6..6.92...18': 0.00099945068359375, '.4.....5...19436....9...3..6...5...21.3...5.68...2...7..5...2....24367...3.....4.': 0.00099945068359375, '.6234.75.1....56..57.....4.....948..4.......6..583.....3.....91..64....7.59.8326.': 0.0009996891021728516, '.53...79...97534..1.......2.9..8..1....9.7....8..3..7.5.......3..76412...61...94.': 0.0009996891021728516, '...6.2...4...5...1.85.1.62..382.671...........194.735..26.4.53.9...2...7...8.9...': 0.0009996891021728516, '.5.8.7.2.6...1..9.7.254...6.7..2.3.15.4...9.81.3.8..7.9...762.5.6..9...3.8.1.3.4.': 0.0009999275207519531, '...........98.51...519.742.29.4.1.65.........14.

In [279]:
# Benchmark hard puzzles
# Source: http://www.norvig.com/top95.txt
DATA = """4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......
52...6.........7.13...........4..8..6......5...........418.........3..2...87.....
6.....8.3.4.7.................5.4.7.3..2.....1.6.......2.....5.....8.6......1....
48.3............71.2.......7.5....6....2..8.............1.76...3.....4......5....
....14....3....2...7..........9...3.6.1.............8.2.....1.4....5.6.....7.8...
......52..8.4......3...9...5.1...6..2..7........3.....6...1..........7.4.......3.
6.2.5.........3.4..........43...8....1....2........7..5..27...........81...6.....
.524.........7.1..............8.2...3.....6...9.5.....1.6.3...........897........
6.2.5.........4.3..........43...8....1....2........7..5..27...........81...6.....
.923.........8.1...........1.7.4...........658.........6.5.2...4.....7.....9.....
6..3.2....5.....1..........7.26............543.........8.15........4.2........7..
.6.5.1.9.1...9..539....7....4.8...7.......5.8.817.5.3.....5.2............76..8...
..5...987.4..5...1..7......2...48....9.1.....6..2.....3..6..2.......9.7.......5..
3.6.7...........518.........1.4.5...7.....6.....2......2.....4.....8.3.....5.....
1.....3.8.7.4..............2.3.1...........958.........5.6...7.....8.2...4.......
6..3.2....4.....1..........7.26............543.........8.15........4.2........7..
....3..9....2....1.5.9..............1.2.8.4.6.8.5...2..75......4.1..6..3.....4.6.
45.....3....8.1....9...........5..9.2..7.....8.........1..4..........7.2...6..8..
.237....68...6.59.9.....7......4.97.3.7.96..2.........5..47.........2....8.......
..84...3....3.....9....157479...8........7..514.....2...9.6...2.5....4......9..56
.98.1....2......6.............3.2.5..84.........6.........4.8.93..5...........1..
..247..58..............1.4.....2...9528.9.4....9...1.........3.3....75..685..2...
4.....8.5.3..........7......2.....6.....5.4......1.......6.3.7.5..2.....1.9......
.2.3......63.....58.......15....9.3....7........1....8.879..26......6.7...6..7..4
1.....7.9.4...72..8.........7..1..6.3.......5.6..4..2.........8..53...7.7.2....46
4.....3.....8.2......7........1...8734.......6........5...6........1.4...82......
.......71.2.8........4.3...7...6..5....2..3..9........6...7.....8....4......5....
6..3.2....4.....8..........7.26............543.........8.15........8.2........7..
.47.8...1............6..7..6....357......5....1..6....28..4.....9.1...4.....2.69.
......8.17..2........5.6......7...5..1....3...8.......5......2..4..8....6...3....
38.6.......9.......2..3.51......5....3..1..6....4......17.5..8.......9.......7.32
...5...........5.697.....2...48.2...25.1...3..8..3.........4.7..13.5..9..2...31..
.2.......3.5.62..9.68...3...5..........64.8.2..47..9....3.....1.....6...17.43....
.8..4....3......1........2...5...4.69..1..8..2...........3.9....6....5.....2.....
..8.9.1...6.5...2......6....3.1.7.5.........9..4...3...5....2...7...3.8.2..7....4
4.....5.8.3..........7......2.....6.....5.8......1.......6.3.7.5..2.....1.8......
1.....3.8.6.4..............2.3.1...........958.........5.6...7.....8.2...4.......
1....6.8..64..........4...7....9.6...7.4..5..5...7.1...5....32.3....8...4........
249.6...3.3....2..8.......5.....6......2......1..4.82..9.5..7....4.....1.7...3...
...8....9.873...4.6..7.......85..97...........43..75.......3....3...145.4....2..1
...5.1....9....8...6.......4.1..........7..9........3.8.....1.5...2..4.....36....
......8.16..2........7.5......6...2..1....3...8.......2......7..3..8....5...4....
.476...5.8.3.....2.....9......8.5..6...1.....6.24......78...51...6....4..9...4..7
.....7.95.....1...86..2.....2..73..85......6...3..49..3.5...41724................
.4.5.....8...9..3..76.2.....146..........9..7.....36....1..4.5..6......3..71..2..
.834.........7..5...........4.1.8..........27...3.....2.6.5....5.....8........1..
..9.....3.....9...7.....5.6..65..4.....3......28......3..75.6..6...........12.3.8
.26.39......6....19.....7.......4..9.5....2....85.....3..2..9..4....762.........4
2.3.8....8..7...........1...6.5.7...4......3....1............82.5....6...1.......
6..3.2....1.....5..........7.26............843.........8.15........8.2........7..
1.....9...64..1.7..7..4.......3.....3.89..5....7....2.....6.7.9.....4.1....129.3.
.........9......84.623...5....6...453...1...6...9...7....1.....4.5..2....3.8....9
.2....5938..5..46.94..6...8..2.3.....6..8.73.7..2.........4.38..7....6..........5
9.4..5...25.6..1..31......8.7...9...4..26......147....7.......2...3..8.6.4.....9.
...52.....9...3..4......7...1.....4..8..453..6...1...87.2........8....32.4..8..1.
53..2.9...24.3..5...9..........1.827...7.........981.............64....91.2.5.43.
1....786...7..8.1.8..2....9........24...1......9..5...6.8..........5.9.......93.4
....5...11......7..6.....8......4.....9.1.3.....596.2..8..62..7..7......3.5.7.2..
.47.2....8....1....3....9.2.....5...6..81..5.....4.....7....3.4...9...1.4..27.8..
......94.....9...53....5.7..8.4..1..463...........7.8.8..7.....7......28.5.26....
.2......6....41.....78....1......7....37.....6..412....1..74..5..8.5..7......39..
1.....3.8.6.4..............2.3.1...........758.........7.5...6.....8.2...4.......
2....1.9..1..3.7..9..8...2.......85..6.4.........7...3.2.3...6....5.....1.9...2.5
..7..8.....6.2.3...3......9.1..5..6.....1.....7.9....2........4.83..4...26....51.
...36....85.......9.4..8........68.........17..9..45...1.5...6.4....9..2.....3...
34.6.......7.......2..8.57......5....7..1..2....4......36.2..1.......9.......7.82
......4.18..2........6.7......8...6..4....3...1.......6......2..5..1....7...3....
.4..5..67...1...4....2.....1..8..3........2...6...........4..5.3.....8..2........
.......4...2..4..1.7..5..9...3..7....4..6....6..1..8...2....1..85.9...6.....8...3
8..7....4.5....6............3.97...8....43..5....2.9....6......2...6...7.71..83.2
.8...4.5....7..3............1..85...6.....2......4....3.26............417........
....7..8...6...5...2...3.61.1...7..2..8..534.2..9.......2......58...6.3.4...1....
......8.16..2........7.5......6...2..1....3...8.......2......7..4..8....5...3....
.2..........6....3.74.8.........3..2.8..4..1.6..5.........1.78.5....9..........4.
.52..68.......7.2.......6....48..9..2..41......1.....8..61..38.....9...63..6..1.9
....1.78.5....9..........4..2..........6....3.74.8.........3..2.8..4..1.6..5.....
1.......3.6.3..7...7...5..121.7...9...7........8.1..2....8.64....9.2..6....4.....
4...7.1....19.46.5.....1......7....2..2.3....847..6....14...8.6.2....3..6...9....
......8.17..2........5.6......7...5..1....3...8.......5......2..3..8....6...4....
963......1....8......2.5....4.8......1....7......3..257......3...9.2.4.7......9..
15.3......7..4.2....4.72.....8.........9..1.8.1..8.79......38...........6....7423
..........5724...98....947...9..3...5..9..12...3.1.9...6....25....56.....7......6
....75....1..2.....4...3...5.....3.2...8...1.......6.....1..48.2........7........
6.....7.3.4.8.................5.4.8.7..2.....1.3.......2.....5.....7.9......1....
....6...4..6.3....1..4..5.77.....8.5...8.....6.8....9...2.9....4....32....97..1..
.32.....58..3.....9.428...1...4...39...6...5.....1.....2...67.8.....4....95....6.
...5.3.......6.7..5.8....1636..2.......4.1.......3...567....2.8..4.7.......2..5..
.5.3.7.4.1.........3.......5.8.3.61....8..5.9.6..1........4...6...6927....2...9..
..5..8..18......9.......78....4.....64....9......53..2.6.........138..5....9.714.
..........72.6.1....51...82.8...13..4.........37.9..1.....238..5.4..9.........79.
...658.....4......12............96.7...3..5....2.8...3..19..8..3.6.....4....473..
.2.3.......6..8.9.83.5........2...8.7.9..5........6..4.......1...1...4.22..7..8.9
.5..9....1.....6.....3.8.....8.4...9514.......3....2..........4.8...6..77..15..6.
.....2.......7...17..3...9.8..7......2.89.6...13..6....9..5.824.....891..........
3...8.......7....51..............36...2..4....7...........6.13..452...........8.."""

# print(benchmark(DATA.split("\n"), recursive_backtrack_solve, conv_data_to_soduko)) # over 32m, no finished run

print(benchmark(DATA.split("\n"), optimized_recursive_backtrack_solve, conv_data_to_soduko)) # ~11m

{'4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......': 10.48803186416626, '52...6.........7.13...........4..8..6......5...........418.........3..2...87.....': 6.817672491073608, '6.....8.3.4.7.................5.4.7.3..2.....1.6.......2.....5.....8.6......1....': 0.15123844146728516, '48.3............71.2.......7.5....6....2..8.............1.76...3.....4......5....': 1.5073890686035156, '....14....3....2...7..........9...3.6.1.............8.2.....1.4....5.6.....7.8...': 226.14238142967224, '......52..8.4......3...9...5.1...6..2..7........3.....6...1..........7.4.......3.': 0.5499556064605713, '6.2.5.........3.4..........43...8....1....2........7..5..27...........81...6.....': 10.718292236328125, '.524.........7.1..............8.2...3.....6...9.5.....1.6.3...........897........': 8.041988849639893, '6.2.5.........4.3..........43...8....1....2........7..5..27...........81...6.....': 6.0271618366241455, '.923.........8.1...........1.7.4...........658.........

In [282]:
# Benchmark hardest puzzles
# Source: http://www.norvig.com/hardest.txt

DATA = """85...24..72......9..4.........1.7..23.5...9...4...........8..7..17..........36.4.
..53.....8......2..7..1.5..4....53...1..7...6..32...8..6.5....9..4....3......97..
12..4......5.69.1...9...5.........7.7...52.9..3......2.9.6...5.4..9..8.1..3...9.4
...57..3.1......2.7...234......8...4..7..4...49....6.5.42...3.....7..9....18.....
7..1523........92....3.....1....47.8.......6............9...5.6.4.9.7...8....6.1.
1....7.9..3..2...8..96..5....53..9...1..8...26....4...3......1..4......7..7...3..
1...34.8....8..5....4.6..21.18......3..1.2..6......81.52..7.9....6..9....9.64...2
...92......68.3...19..7...623..4.1....1...7....8.3..297...8..91...5.72......64...
.6.5.4.3.1...9...8.........9...5...6.4.6.2.7.7...4...5.........4...8...1.5.2.3.4.
7.....4...2..7..8...3..8.799..5..3...6..2..9...1.97..6...3..9...3..4..6...9..1.35
....7..2.8.......6.1.2.5...9.54....8.........3....85.1...3.2.8.4.......9.7..6...."""

# print(benchmark(DATA.split("\n"), recursive_backtrack_solve, conv_data_to_soduko))

print(benchmark(DATA.split("\n"), optimized_recursive_backtrack_solve, conv_data_to_soduko)) # ~ 0.9s

{'...92......68.3...19..7...623..4.1....1...7....8.3..297...8..91...5.72......64...': 0.002502918243408203, '7..1523........92....3.....1....47.8.......6............9...5.6.4.9.7...8....6.1.': 0.0035097599029541016, '1...34.8....8..5....4.6..21.18......3..1.2..6......81.52..7.9....6..9....9.64...2': 0.004998922348022461, '7.....4...2..7..8...3..8.799..5..3...6..2..9...1.97..6...3..9...3..4..6...9..1.35': 0.013012170791625977, '85...24..72......9..4.........1.7..23.5...9...4...........8..7..17..........36.4.': 0.023575544357299805, '..53.....8......2..7..1.5..4....53...1..7...6..32...8..6.5....9..4....3......97..': 0.037056684494018555, '1....7.9..3..2...8..96..5....53..9...1..8...26....4...3......1..4......7..7...3..': 0.040041208267211914, '12..4......5.69.1...9...5.........7.7...52.9..3......2.9.6...5.4..9..8.1..3...9.4': 0.09720873832702637, '...57..3.1......2.7...234......8...4..7..4...49....6.5.42...3.....7..9....18.....': 0.10610318183898926, '.6.5.4.3.1...9...8.........9...5...6